In [1]:
from openai import OpenAI

from gimkit import Query
from gimkit import guide as g

In [2]:
# Define the llm_request using any framework you like

GIM_MODEL_NAME = "artifacts/09191-gim-sft-2/sft-gim"
OPEN_API_KEY = "EMPTY"
OPENAI_API_BASE = "http://localhost:8000/v1"


def llm_request(query: str | Query) -> str:
    if isinstance(query, Query):
        query = str(query)
    client = OpenAI(
        api_key=OPEN_API_KEY,
        base_url=OPENAI_API_BASE,
    )
    completion = client.completions.create(
        model=GIM_MODEL_NAME,
        prompt=query,
        temperature=0.0,
        max_tokens=4096,
        presence_penalty=1,
        seed=0,
    )
    return completion.choices[0].text

In [3]:
def print_result_and_tags(question: str):
    query = Query(question)
    response = llm_request(query)
    result = query.infill(response)

    # Print the infilled result
    print(result)

    print("\n" + "=" * 40 + "\n")

    # Print each tag's predicted content
    for tag in result.tags:
        print(tag)

# A 24 Puzzle QA

In [4]:
question = """2 {} 4 {} 6 {} 2 = 24""".format(*[g(desc="a single math operator") for _ in range(3)])

print_result_and_tags(question)

2 × 4 × 6 ÷ 2 = 24


<|MASKED id="m_0" desc="a single math operator"|>×<|/MASKED|>
<|MASKED id="m_1" desc="a single math operator"|>×<|/MASKED|>
<|MASKED id="m_2" desc="a single math operator"|>÷<|/MASKED|>


# A Chinese Chain-of-Thought QA

In [5]:
question = """对于淀粉和纤维素两种物质，下列说法正确的是____
A. 两者都能水解，且水解的最终产物相同
B. 两者含C、H、O三种元素的质量分数相同，且互为同分异构体
C. 它们都属于糖类，且都是溶于水的高分子化合物
D. 都可 用$\\left ( C_6H_6O_5\\right )_n$表示，但淀粉能发生银镜反应，而纤维素不能

让我们一步步思考
1. {}
2. {}
3. {}
4. {}
5. {}
6. {}
所以答案是：{}
""".format(
    *([g(desc="一个思考步骤") for _ in range(6)] + [g.options(choices=["A", "B", "C", "D"])])
)

print_result_and_tags(question)

对于淀粉和纤维素两种物质，下列说法正确的是____
A. 两者都能水解，且水解的最终产物相同
B. 两者含C、H、O三种元素的质量分数相同，且互为同分异构体
C. 它们都属于糖类，且都是溶于水的高分子化合物
D. 都可 用$\left ( C_6H_6O_5\right )_n$表示，但淀粉能发生银镜反应，而纤维素不能

让我们一步步思考
1. 淀粉和纤维素都是多糖，它们的水解最终产物都是葡萄糖，因此选项A是正确的。
2. 虽然淀粉和纤维素都含有C、H、O三种元素，但它们的分子式不同，因此不能互为同分异构体。选项B是错误的。
3. 淀粉和纤维素都是糖类，但它们不溶于水，因此选项C是错误的。
4. 淀粉和纤维素的分子式可以表示为$\left ( C_6H_{10}O_5\right )_n$，而不是$\left ( C_6H_6O_5\right )_n$。此外，淀粉能发生银镜反应，而纤维素不能，因此选项D是错误的。
5. 综上所述，只有选项A是正确的。
6. 因此，正确答案是A。
所以答案是：A



<|MASKED id="m_0" desc="一个思考步骤"|>淀粉和纤维素都是多糖，它们的水解最终产物都是葡萄糖，因此选项A是正确的。<|/MASKED|>
<|MASKED id="m_1" desc="一个思考步骤"|>虽然淀粉和纤维素都含有C、H、O三种元素，但它们的分子式不同，因此不能互为同分异构体。选项B是错误的。<|/MASKED|>
<|MASKED id="m_2" desc="一个思考步骤"|>淀粉和纤维素都是糖类，但它们不溶于水，因此选项C是错误的。<|/MASKED|>
<|MASKED id="m_3" desc="一个思考步骤"|>淀粉和纤维素的分子式可以表示为$\left ( C_6H_{10}O_5\right )_n$，而不是$\left ( C_6H_6O_5\right )_n$。此外，淀粉能发生银镜反应，而纤维素不能，因此选项D是错误的。<|/MASKED|>
<|MASKED id="m_4" desc="一个思考步骤"|>综上所述，只有选项A是正确的。<|/MASKED|>
<|MASKED id="m_5" desc="一个思考步骤"|>因此，正确答案是A。<|/MASKED|>
<|MASKED id="m_6" de

# Data synthesis

In [6]:
question = f"""# A short article

This small ebook is here to teach you a programming language called Forth. Forth is a language unlike most others. It’s not functional or object oriented, it doesn’t have type-checking, and it basically has zero syntax. It was written in the 70s, but is still used today for certain applications.

Why would you want to learn such an odd language? Every new programming language you learn helps you think about problems in new ways. Forth is very easy to learn, but it requires you to think in a different way than you’re used to. That makes it a perfect language to broaden your coding horizons.

This book includes a simple implementation of Forth I wrote in JavaScript. It’s by no means perfect, and is missing a lot of the functionality you’d expect in a real Forth system. It’s just here to give you an easy way to try out the examples. (If you’re a Forth expert, please contribute here and make it better!)

I’m going to assume that you know at least one other programming language, and have a basic idea of how stacks work as a data structure.

# Dependent Questions and Answers

Q: {g("A easy question about the article.")}
A: {g("A concise answer to the question.")}

---

Q: {g("A hard question about the article.")}
A: {g("A detailed answer to the question.")}

---

Q: {g("A very hard question about the article.")}
A: {g("A very detailed answer to the question.")}

# Note

- The questions should be about the article above.
- The answers should be based on the content of the article above.
- The questions are dependent, i.e., the answer to the first question may be needed to answer the second question, and so on.
"""

print_result_and_tags(question)

# A short article

This small ebook is here to teach you a programming language called Forth. Forth is a language unlike most others. It’s not functional or object oriented, it doesn’t have type-checking, and it basically has zero syntax. It was written in the 70s, but is still used today for certain applications.

Why would you want to learn such an odd language? Every new programming language you learn helps you think about problems in new ways. Forth is very easy to learn, but it requires you to think in a different way than you’re used to. That makes it a perfect language to broaden your coding horizons.

This book includes a simple implementation of Forth I wrote in JavaScript. It’s by no means perfect, and is missing a lot of the functionality you’d expect in a real Forth system. It’s just here to give you an easy way to try out the examples. (If you’re a Forth expert, please contribute here and make it better!)

I’m going to assume that you know at least one other programming la

# Controllable text generation

In [7]:
question = "On a screen, mixing #{} and #{} results in #{}.".format(
    g(desc="Green's hexadecimal color code"),
    g(desc="Blue's hexadecimal color code"),
    g(desc="The final hexadecimal color code"),
)

print_result_and_tags(question)

On a screen, mixing #00FF00 and #0000FF results in #00FFFF.


<|MASKED id="m_0" desc="Green&#x27;s hexadecimal color code"|>00FF00<|/MASKED|>
<|MASKED id="m_1" desc="Blue&#x27;s hexadecimal color code"|>0000FF<|/MASKED|>
<|MASKED id="m_2" desc="The final hexadecimal color code"|>00FFFF<|/MASKED|>


# Structured text generation

In [8]:
question = f"""Complete the algorithm profile in TOML:

[algorithm_profile]
name = {g(desc="an any algorithm name")}
type = {g(desc="list[str] type; 4 items")}
performance.time_complexity: {g(desc="str type; use LaTeX expression")},
performance.space_complexity: {g(desc="str type; use LaTeX expression")}"""

print_result_and_tags(question)

Complete the algorithm profile in TOML:

[algorithm_profile]
name = QuickSort
type = sorting, searching
performance.time_complexity: O(n log n),
performance.space_complexity: O(log n)


<|MASKED id="m_0" desc="an any algorithm name"|>QuickSort<|/MASKED|>
<|MASKED id="m_1" desc="list[str] type; 4 items"|>sorting, searching<|/MASKED|>
<|MASKED id="m_2" desc="str type; use LaTeX expression"|>O(n log n)<|/MASKED|>
<|MASKED id="m_3" desc="str type; use LaTeX expression"|>O(log n)<|/MASKED|>


# Code Completion

In [9]:
# From (Fried, et al. 2022) InCoder

question = f'''def count_words(filename):
    """{g(desc="one-line docstring")}"""
    counts = Counter()
    with open(filename) as file:
        for line in file:
            words = line.split(' ')
            counts.update(words)
    return counts
'''

print_result_and_tags(question)

def count_words(filename):
    """Count the number of words in a file."""
    counts = Counter()
    with open(filename) as file:
        for line in file:
            words = line.split(' ')
            counts.update(words)
    return counts



<|MASKED id="m_0" desc="one-line docstring"|>Count the number of words in a file.<|/MASKED|>


In [10]:
# Also from (Fried, et al. 2022) InCoder

question = f'''
def {g()}:
    """ Count the number of occurrences of each word in the file. """
    {g()}
'''

print_result_and_tags(question)


def count_words(file):
    """ Count the number of occurrences of each word in the file. """
    with open(file, 'r') as f:
    words = f.read().split()
    word_count = {}
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count



<|MASKED id="m_0"|>count_words(file)<|/MASKED|>
<|MASKED id="m_1"|>with open(file, 'r') as f:
    words = f.read().split()
    word_count = {}
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count<|/MASKED|>


# Table Completion

In [11]:
question = f"""
| Week | Milk (liters) | Bread (loaves) | Eggs (dozen) |
| ---- | ------------- | -------------- | ------------ |
| 1    | 6             | 4              | 2            |
| 2    | 6             | 5              | 3            |
| 3    | {g()}         | 4              | 2            |
| 4    | 6             | {g()}          | 3            |
"""

print_result_and_tags(question)


| Week | Milk (liters) | Bread (loaves) | Eggs (dozen) |
| ---- | ------------- | -------------- | ------------ |
| 1    | 6             | 4              | 2            |
| 2    | 6             | 5              | 3            |
| 3    | 6         | 4              | 2            |
| 4    | 6             | 5          | 3            |



<|MASKED id="m_0"|>6<|/MASKED|>
<|MASKED id="m_1"|>5<|/MASKED|>


# KG Extraction

In [12]:
question = f"""## Content

This small ebook is here to teach you a programming language called Forth. Forth is a language unlike most others. It’s not functional or object oriented, it doesn’t have type-checking, and it basically has zero syntax. It was written in the 70s, but is still used today for certain applications.

## Extraction

Extract knowledge graph triplets (head, relation, tail) from the content.

1. ({g()}, {g()}, {g()})
2. ({g()}, {g()}, {g()})
3. ({g()}, {g()}, {g()})
4. ({g()}, {g()}, {g()})
5. ({g()}, {g()}, {g()})
6. ({g()}, {g()}, {g()})
"""

print_result_and_tags(question)

## Content

This small ebook is here to teach you a programming language called Forth. Forth is a language unlike most others. It’s not functional or object oriented, it doesn’t have type-checking, and it basically has zero syntax. It was written in the 70s, but is still used today for certain applications.

## Extraction

Extract knowledge graph triplets (head, relation, tail) from the content.

1. (Forth, is a language, unlike most others)
2. (Forth, was written in, the 70s)
3. (Forth, is still used today for, certain applications)
4. (Forth, has zero syntax, It was written in the 70s, but is still used today for certain applications.)
5. (Forth, doesn't have, type-checking)
6. (Forth, doesn't have, functional or object oriented)



<|MASKED id="m_0"|>Forth<|/MASKED|>
<|MASKED id="m_1"|>is a language<|/MASKED|>
<|MASKED id="m_2"|>unlike most others<|/MASKED|>
<|MASKED id="m_3"|>Forth<|/MASKED|>
<|MASKED id="m_4"|>was written in<|/MASKED|>
<|MASKED id="m_5"|>the 70s<|/MASKED|>
<|MASKE